# import

In [1]:
import pandas as pd
from tqdm.notebook import tqdm as tqdm
import time
from graphviz import *
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
import fasttext as ft
# import importlib
# importlib.reload(hoge)
from my_library.horse import *

# path

In [2]:
path_ubu = '/home/hipro/デスクトップ/Horse/Data/20_21'
path_mac2 = '/Users/rince/Desktop/Horse/Data/saishin2/'
path_mac = '/Users/rince/Desktop/Horse/Data/saishin/'
path_win = '/Users/Owner/Desktop/program/Horse/Data/saishin/'
path_win2 = '/Users/Owner/Desktop/program/Horse/Data/saishin2/'

# load data

In [3]:
results = load_csv(path_mac+'results.csv')
horse_results = load_csv(path_mac+'horse_results.csv')
peds = load_csv(path_mac+'peds.csv')
# 何回やってもロードすると, nanが出る
peds.fillna('nan',inplace=True)
return_tables = load_csv(path_mac+'return.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

# race_id 命名規則

race_id 202105040802\
yyyy_pp_xx_xxrr\
y : year\
p : palce\
x : 謎\
r : race番号

In [17]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                'lambdarank_truncation_level': 29,
                'learning_rate': 0.021772987388503244,
                'num_iterations': 93,
                'num_leaves': 40
            }

In [18]:
ll = LearnLGBM(peds,results,horse_results)
ll.learn_lgb()


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.073375 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46022
[LightGBM] [Info] Number of data points in the train set: 85884, number of used features: 172


In [11]:
# ll = LearnLGBM(peds,results,horse_results)
# ll.get_train_data()

train_query = ll.x_train.groupby(ll.x_train.index).size()
val_x = ll.x_test.iloc[:-10000]
val_y = ll.y_test.iloc[:-10000]
test_query = val_x.groupby(val_x.index).size()
train = lgb.Dataset(ll.x_train, ll.y_train, group=train_query)
valid = lgb.Dataset(val_x, val_y, reference=train, group=test_query)

# 正解ラベル　変えた

In [ ]:
def split_data(df, test_size=0.2, rank_learning=True):
    """
    データを学習データと, 訓練データに分ける関数
    """
    df_ = df.copy()
    df_['rank'] = df_['rank'].map(lambda x: int((1/x)*10) if x<4 else(0))
        
    sorted_id_list = df_.sort_values("date").index.unique()
    train_id_list = sorted_id_list[: round(len(sorted_id_list) * (1 - test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)) :]
    train = df_.loc[train_id_list]#.drop(['date'], axis=1)
    test = df_.loc[test_id_list]#.drop(['date'], axis=1)
    return train, test

    
train, test = split_data(ll.r.data_c.fillna(0),test_size=0.2,rank_learning=False)
x_train = train.drop(['rank', 'date','単勝'], axis=1)
y_train = train['rank']
x_test = test.drop(['rank', 'date','単勝'], axis=1)
y_test = test['rank']
train_query = x_train.groupby(x_train.index).size()
train = lgb.Dataset(x_train, y_train, group=train_query)
val_y = y_test.iloc[:-10000]
test_query = x_test.groupby(x_test.index).size()
valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

In [ ]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                'lambdarank_truncation_level': 10,
                'learning_rate': 0.02273417953255777,
                'num_iterations': 97,
                'num_leaves': 42
            }

{'lambdarank_truncation_level': 10,
 'learning_rate': 0.02273417953255777,
 'num_iterations': 97,
 'num_leaves': 42}

In [13]:
range(1,50)[28]

29

In [ ]:
# {'lambdarank_truncation_level': range(1,50)[33], 'learning_rate': 0.09608503291165846, 'num_iterations': range(50,120)[14], 'num_leaves': range(20,50)[2]}

{'lambdarank_truncation_level': 28, 'learning_rate': 0.021772987388503244}
{'lambdarank_truncation_level':  range(1,50)[28], 'learning_rate': 0.02273417953255777, 'num_iterations': range(50,120)[47], 'num_leaves': range(20,50)[22]}

In [ ]:
ll.learn_lgb2(train,lgbm_params=lgbm_params)

In [10]:
def score(params):
    print("Training start:")

    N_boost_round = []
    Score = []

    lgb_results={}  #履歴格納用
    # train = lgb.Dataset(x_train, y_train, group=train_query)
    # valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

    
    lgb_clf = lgb.train(
        params,
        train,
        num_boost_round=1000,
        valid_sets=valid,
        valid_names=['valid'],
        early_stopping_rounds=20,
        verbose_eval=5,
        evals_result=lgb_results,
    )
#     return lgb_results
    return {'loss': -1.0 * lgb_results['valid']['ndcg@3'][lgb_clf.best_iteration], 'status': STATUS_OK}

def optimize(trials):
#探索スペース
    space = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        # 上位三着を考慮する
        'ndcg_eval_at': [1,2,3],
#         best paramsの返り値は, choiceだとindexか?
        'lambdarank_truncation_level': hp.choice('lambdarank_truncation_level',range(1,50)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        # n_estimators == num_iterations
        # 'num_iterations': hp.choice('num_iterations',range(50,120)),
        # 'num_leaves': hp.choice('num_leaves',range(20,50)),
        'boosting_type': 'gbdt',
        'random_state': 777
    }

    max_evals = 25      #探索回数(25くらいで十分)
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    print("best parameters:", best)

In [12]:
trials = Trials()
optimize(trials)

Training start:                                       
[LightGBM] [Warning]                                  
Auto-choosing col-wise multi-threading, the overhead of testing was 0.094964 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info]                                     
Total Bins 46022                                      
[LightGBM] [Info]                                     
Number of data points in the train set: 85884, number of used features: 172
Training until validation scores don't improve for 20 rounds
  0%|          | 0/25 [00:02<?, ?trial/s, best loss=?]

/Users/rince/Library/Python/3.7/lib/python/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')

/Users/rince/Library/Python/3.7/lib/python/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))



[5]	valid's ndcg@1: 0.507126	valid's ndcg@2: 0.492156	valid's ndcg@3: 0.489594
[10]	valid's ndcg@1: 0.507126	valid's ndcg@2: 0.490826	valid's ndcg@3: 0.490807
[15]	valid's ndcg@1: 0.510689	valid's ndcg@2: 0.496977	valid's ndcg@3: 0.493086
[20]	valid's ndcg@1: 0.510689	valid's ndcg@2: 0.499564	valid's ndcg@3: 0.494827
Early stopping, best iteration is:                    
[1]	valid's ndcg@1: 0.538005	valid's ndcg@2: 0.512396	valid's ndcg@3: 0.505089
Training start:                                                                  
[LightGBM] [Warning]                                                             
Auto-choosing col-wise multi-threading, the overhead of testing was 0.181499 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info]                                                                
Total Bins 46022                                                                 
[LightGBM] [Info]                                                          

# Simulate


In [20]:

start_time = time.time()
for odds in [1.1,2.0,3.0,4.0,5.0,6.0,7.0]:
    print()
    sl = RankSimulater(ll.model)
    print("odds",odds)
    sl.calc_tansho(ll.r.data_c.iloc[-10000:],return_tables,odds=odds)
    print("time", time.time() - start_time)




odds 1.1
not_bet_count 0
---------------------
単勝
的中率 : 0 / 718
的中% : 0.00 %
収支   : -6020.0 円
的中レース : {'01': 18, '02': 16, '03': 16, '04': 17, '05': 14, '06': 14, '07': 21, '08': 13, '09': 5, '10': 16, '11': 7, '12': 16}
time 17.107146978378296

odds 2.0
not_bet_count 95
---------------------
単勝
的中率 : 0 / 623
的中% : 0.00 %
収支   : -4480.0 円
的中レース : {'01': 18, '02': 16, '03': 16, '04': 17, '05': 14, '06': 14, '07': 21, '08': 13, '09': 5, '10': 16, '11': 7, '12': 16}
time 34.02035689353943

odds 3.0
not_bet_count 233
---------------------
単勝
的中率 : 0 / 485
的中% : 0.00 %
収支   : -1770.0 円
的中レース : {'01': 18, '02': 16, '03': 16, '04': 17, '05': 14, '06': 14, '07': 21, '08': 13, '09': 5, '10': 16, '11': 7, '12': 16}
time 50.90834403038025

odds 4.0
not_bet_count 362
---------------------
単勝
的中率 : 0 / 356
的中% : 0.00 %
収支   : -370.0 円
的中レース : {'01': 18, '02': 16, '03': 16, '04': 17, '05': 14, '06': 14, '07': 21, '08': 13, '09': 5, '10': 16, '11': 7, '12': 16}
time 67.90319395065308

odds 5.0
not_b

In [ ]:
sl = RankSimulater(ll.model)
sl.calc_sanrenpuku(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

In [ ]:
sl = RankSimulater(ll.model)
sl.calc_sanrentan(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

# race_id_list

In [6]:


race_id_list = ['2022050208{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022050206{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022070304{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022070302{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022040104{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022040102{}'.format(str(i).zfill(2)) for i in range(1,13)]



In [ ]:
    
class Predictor(LearnLGBM):


    def __init__(self,peds,results,horse_results,race_id_list):
        super(Predictor, self).__init__(peds,results,horse_results)
        self.race_id_list = race_id_list


    def process_data(self):
        race_id_list = self.race_id_list.copy()
        data =  ShutubaTable.scrape(race_id_list, self.date)
        self.data = data
        peds = self.peds.copy()
        results = self.results.copy()
        horse_results = self.horse_results.copy()


        nopeds_id_list = []
        for ind in data['horse_id'].astype(int).unique():
            if ind not in peds.index:
                nopeds_id_list.append(str(ind))

        if len(nopeds_id_list)!=0:
            peds_tmp = Peds.scrape(nopeds_id_list)
            pe_tmp = Peds(peds_tmp)
            pe_tmp.regularize_peds() 
            new_peds = update_data(peds, pe_tmp.peds_re)
        else:
            new_peds = peds.copy()
        self.peds = new_peds.copy()
        path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
        new_peds.to_csv(path_ft,header=False,index=False,sep=',')
        self.learn_model_ft(path_ft=path_ft)
        model_ft = self.get_model_ft()

        pe = Peds(new_peds)
        pe.regularize_peds()
        pe.vectorize(pe.peds_re,model_ft)
        self.pe = pe
        print("pe finish")
        print("pe regularizrd")

        r = Results(results)
        r.preprocessing()
        horse_id_list = data['horse_id'].astype(str).unique()
        horse_results_tmp = HorseResults.scrape(horse_id_list)
        new_horse_results = update_data(horse_results,horse_results_tmp)
        self.hores_results = new_horse_results.copy()
        hr = HorseResults(new_horse_results)
        self.hr = hr
        print("hr finish")

        r.merge_horse_results(hr)
        r.merge_peds(pe.peds_vec)
        r.process_categorical()  
        self.r = r
    
    
    def predict(self, race_id):
        data =  ShutubaTable.scrape([str(race_id)], self.date)
        st = ShutubaTable(data)
        st.preprocessing()
        st.merge_horse_results(self.hr)
        st.merge_peds(self.pe.peds_vec)
        st.process_categorical(self.r.le_horse, self.r.le_jockey, self.r.data_pe)
        self.st = st
        sl = RankSimulater(self.model)
        pred_table = sl.return_pred_table(st.data_c)
        self.sl = sl
        print(pred_table)
            

    def show_results_today(self):
        data =  ShutubaTable.scrape(self.race_id_list, self.date)
        self.data = data.copy()
        st = ShutubaTable(self.data)
        st.preprocessing()
        st.merge_horse_results(self.hr)
        st.merge_peds(self.pe.peds_vec)
        st.process_categorical(self.r.le_horse, self.r.le_jockey, self.r.data_pe)
        sl = RankSimulater(self.model)
        sl.return_table_today(self.race_id_list)
        sl.show_results_today(st ,self.race_id_list)

In [15]:
pt = Predictor(peds,results,horse_results,race_id_list)
pt.learn_lgb(lgbm_params=lgbm_params)


pe finish
pe regularizrd



hr finish


/Users/rince/Library/Python/3.7/lib/python/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.067707 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46026
[LightGBM] [Info] Number of data points in the train set: 85884, number of used features: 172


# 新潟

In [ ]:
base  = '2022040101'
pt.predict(base+'05')

# 東京

In [ ]:
base  = '2022050205'
pt.predict(base+'05')

# 中京

In [ ]:
base  = '2022070301'
pt.predict(base+'05')

In [16]:
pt.show_results_today()


-------------------
predict
              馬番    scores
202205020801   1 -0.858855
202205020801   2 -0.858855
202205020801   3 -0.858855
actual
                0            1                 2            3
202205020801   単勝            6              140円          1人気
202205020801   複勝        6 9 5      100円670円130円    1人気9人気2人気
202205020801   枠連          5 6            4,050円          9人気
202205020801   馬連          6 9            4,530円         12人気
202205020801  ワイド  6 9 5 6 5 9  1,150円130円2,440円  13人気1人気22人気
202205020801   馬単          6 9            5,770円         15人気
202205020801  3連複        5 6 9            3,620円         12人気
202205020801  3連単        6 9 5           20,450円         56人気
-------------------
predict
              馬番    scores
202205020802   9 -0.857867
202205020802  10 -0.857867
202205020802   1 -0.858855
actual
                0              1             2            3
202205020802   単勝              1          430円          2人気
202205020802   複勝         1 7 11  1

# 日付に注意

In [ ]:
date = '2022/12/31'

# Results scraping

In [ ]:
# race_id_list = results.index.astype('str')

results_tmp = Results.scrape(race_id_list)

# Horse_results scraping

In [ ]:
horse_id_list = results_tmp['horse_id'].astype(str).unique()
horse_results_tmp = HorseResults.scrape(horse_id_list)
# save_path = '/Users/rince/Desktop/Horse/Data/horse_2020.csv'

# Peds scraping

In [ ]:
peds_2021 = Peds.scrape(horse_id_list)
pe_2021 = Peds(peds_2021)
pe_2021.regularize_peds()
peds_tmp = pe_2021.peds_re

# Return scraping

In [ ]:
returns_tmp = Return.scrape(race_id_list)

# update

In [ ]:
new_results = update_data(load_csv(path_mac+'results.csv'),results_tmp)
new_horse_results = update_data(load_csv(path_mac+'horse_results.csv'),horse_results_tmp)
new_peds = update_data(load_csv(path_mac+'peds.csv'),peds_tmp)
new_return = update_data(load_csv(path_mac+'return.csv'),returns_tmp)

# save

In [ ]:
new_results.to_csv(path_mac+'results.csv')
new_horse_results.to_csv(path_mac+'horse_results.csv')
new_peds.to_csv(path_mac+'peds.csv')
new_return.to_csv(path_mac+'return.csv')

# 重要度

In [ ]:
importances = pd.DataFrame(
{'features' : x_train.columns, 'importances' : lgb_rank.feature_importance()})
print(importances.sort_values('importances', ascending=False)[:30])

# fastText

流れ
1. fasttext用の血統データの学習データを作る (血統の情報のみ, index ヘッダはいらない)
2. fasttext学習
3. 学習モデルを使って, 血統データをベクトル化
4. ベクトル化して r.data_cに concat
5. 学習


教師あり, 教師なしでも生成されるベクトルは等しい

# model_ft 作成

In [ ]:
# 相対パスしかできない
# dim : 出力の次元
# minn : n_gramの最小単位
# maxn : n_gramの最大単位
path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
# 上書き保存OK
peds.to_csv(path_ft,header=False,index=False,sep=',')
model_ft = ft.train_unsupervised(path_ft,dim=62,minn=2,maxn=14)

model[model.words[1]] と model.get_input_vector(ind=1) は等価

# rank = [int((1/i)*10)] と変換してみてはどうか